In [1]:
# Import essential modules
import numpy as np
import pandas as pd
import itertools; import joblib; import time
from scipy import linalg

In [2]:
# Set pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Import custom module###
import module.predict_by_reservoir_single as lrt

In [4]:
# ----------------------------------------------- #
# Prepare data and set parameters
# ----------------------------------------------- #
# Load time series
lorenz = pd.read_csv('./data/lorenz_100.csv')

# Prepare output dataframe
output_all_df = pd.DataFrame()

# Set target variable###
target_var, target_ts = 'z', lorenz[90:]

# Netrowk parameters###
num_reservoir_nodes = 5 # Equals to the number of variables in the BZ reaction
#num_input_nodes = 1 # Input is a scalar value
#num_output_nodes = 1 # Output is a scalar value

# Specify the test fraction
test_fraction = 0.2

In [5]:
# Paratemers of the model###
epsilon0 = 0.0667
q0 = 0.0008
gamma0 = 0.0000889
f0 = 0.65
alfa0 = 270
beta0 = 430
#betap0 = 412.8
#kappa0 = 0.000099 #####

In [6]:
# ----------------------------------------------- #
# Search w_in_sparsity and w_in_strength
# ----------------------------------------------- #
# For parallel analysis###
def reservoir_computing_parallel(w_in_sparsity = 0.1, w_in_strength = 0.1,
                                 epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0):
    # Step.1: Initialize class "SimplexReservoir"
    par_lrc = lrt.Reservoir(network_name = "BZ Reaction")
    # Step.2: Prepare target data
    par_lrc.prepare_target_data(target_ts, target_var, test_fraction)
    # Step.3: Initialize reservoir###
    par_lrc.initialize_reservoir(num_reservoir_nodes = 5, w_in_sparsity = w_in_sparsity, w_in_strength = w_in_strength)
    # Step.4: Training reservoir###
    par_lrc.compute_reservoir_state(epsilon = epsilon, q = q, gamma = gamma, f = f, 
                                  alfa = alfa, beta = beta)
    # Step.5: Learn weights by ridge regression
    par_lrc.learn_model(ridge_lambda = 0.05, washout_fraction = 0.05)
    # Step.6: Predict test data
    par_lrc.predict()
    # Step.7: Summarize stats
    par_lrc.summarize_stat()
    return par_lrc.result_summary_df
    # ------------------------------------------------------------------------------------------ #

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Parameter search
WinSp_range = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
WinSt_range = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

par_rc1 = joblib.Parallel(n_jobs=-3, verbose = 10)([joblib.delayed(reservoir_computing_parallel)
                                    (w_in_sparsity = x, w_in_strength = y) for x, y in itertools.product(WinSp_range, WinSt_range)])
output_all_df1 = par_rc1[0]
for j in range(1,len(par_rc1)): output_all_df1 = output_all_df1.append(par_rc1[j])
output_all_df1 = output_all_df1.reset_index(drop = True)
wspst_min = output_all_df1.loc[output_all_df1['NMSE_test'].min() == output_all_df1['NMSE_test']]
w_in_sp_best = float(np.array(wspst_min["Win_sparsity"])[0])
w_in_st_best = float(np.array(wspst_min["Win_strength"])[0])

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   1 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-3)]: Done  12 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-3)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-3)]: Done  53 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-3)]: Done  68 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-3)]: Done  85 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-3)]: Done 118 out of 160 | elapsed:   26.0s remaining:    9.2s
[Parallel(n_jobs=-3)]: Done 135 out of 160 | elapsed:   28.8s remaining:    5.2s
[Parallel(n_jobs=-3)]: Done 152 out of 160 | elapsed:   32.2s remaining:    1.6s
[Parallel(n_jobs=-3)]: Done 160 out of 160 | elapsed:   32.5s finished


## Results for coupled oscillators
### Variable $x$
#### Results with in/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9059,0.8917,0.0968,0.104243352,0.0328671,0.0337291,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.2,991,0.3,0,0.01,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9068,0.8928,0.0965,0.104172935,0.0326897,0.0336836,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.02,0.03,0.01


#### Results with in/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9055,0.891,0.097,0.104601789,0.0330325,0.0339615,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.2,991,0.3,0,0.01,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9063,0.892,0.0968,0.104617702,0.0328984,0.0339718,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.03,0.0


#### Results with in/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9094,0.8965,0.0949,0.101614844,0.0316345,0.0320496,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.3,991,0.3,0,0.02,0.01,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9099,0.8971,0.0948,0.101879476,0.0315642,0.0322167,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.02,0.0


#### Results with out/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.907,0.8928,0.0962,0.103664806,0.0324703,0.0333558,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.3,991,0.3,0,0.02,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9083,0.8943,0.0958,0.103455407,0.0321866,0.0332211,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.02,0.03,0.0


#### Results with out/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.906,0.8916,0.0967,0.104304121,0.0328222,0.0337684,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.2,991,0.3,0,0.02,0.01,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.907,0.8929,0.0964,0.104129482,0.0326212,0.0336555,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.01,0.02


#### Results with anti/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.907,0.8927,0.0962,0.10371686,0.032491,0.0333893,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.2,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9082,0.8942,0.0958,0.103485491,0.0322136,0.0332405,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.01,0.0


#### Results with out/in-phase initial condition

In [9]:
# k = 0.00002475 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9083,0.8941,0.0955,0.102903194,0.0320228,0.0328674,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.3,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9094,0.8955,0.0952,0.102820981,0.0317889,0.0328149,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.02,0.03,0.01


#### Results with anti/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9085,0.8942,0.0954,0.102909189,0.0319627,0.0328713,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.3,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9095,0.8956,0.0951,0.102781367,0.0317449,0.0327897,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.02,0.01


#### Results with anti/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9079,0.8937,0.0957,0.103147236,0.0321557,0.0330235,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.08,0.2,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9091,0.8951,0.0953,0.103002949,0.0318932,0.0329312,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.08,0.1,991,0.3,0,0.01,0.02,0.01


### Variable $y$
#### Results with in/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8184,0.7958,0.1158,0.127093076,0.0525199,0.0567237,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.3,991,0.3,0,0.01,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8159,0.7937,0.1151,0.125641357,0.0519114,0.0554352,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.02,0.01


#### Results with in/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8179,0.7945,0.1161,0.127599099,0.0527839,0.0571763,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.3,991,0.3,0,0.01,0.02,0.01


In [10]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8156,0.7931,0.1152,0.125895623,0.0519791,0.0556598,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.02,0.0


#### Results with in/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8248,0.8049,0.1133,0.123377564,0.0502331,0.0534556,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.2,991,0.3,0,0.02,0.01,0.0


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8336,0.8098,0.1092,0.119564082,0.0466637,0.0502021,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.03,0.02,0.02


#### Results with out/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8197,0.7969,0.1154,0.126751238,0.0521253,0.056419,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.2,991,0.3,0,0.02,0.01,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.811,0.7903,0.1169,0.126432891,0.053533,0.0561359,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.05,0.0


#### Results with out/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8184,0.7953,0.1159,0.127358581,0.0525877,0.0569609,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.3,991,0.3,0,0.02,0.02,0.0


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8148,0.7921,0.1155,0.126228891,0.0522435,0.0559549,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.07,0.0


#### Results with anti/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8198,0.7967,0.1154,0.126837569,0.0521297,0.0564958,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.2,991,0.3,0,0.02,0.02,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8062,0.7815,0.1185,0.129224435,0.0549527,0.0586421,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with out/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8216,0.799,0.1147,0.125920248,0.051483,0.0556816,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.3,991,0.3,0,0.01,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8221,0.8008,0.1129,0.123167706,0.0499008,0.0532739,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.03,0.01


#### Results with anti/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8214,0.7986,0.1148,0.126119166,0.0515636,0.0558577,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.2,991,0.3,0,0.02,0.04,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8204,0.7984,0.1136,0.124106324,0.0504985,0.0540889,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.03,0.02,0.01


#### Results with anti/out-of-phase initial condition

In [9]:
# k = 0.00002475 # This should be for z 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.785,0.7789,0.1493,0.16334116,0.0818404,0.1000006,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8224,0.8007,0.1129,0.123322864,0.0498985,0.0534082,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.1,0.0,991,0.3,0,0.02,0.02,0.01


### Variable $z$
#### Results with in/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7851,0.7787,0.1492,0.163287573,0.0816814,0.099935,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.01,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.793,0.796,0.1411,0.150307115,0.0730744,0.084678,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.01,0.03,0.0


#### Results with in/anti-phase initial condition

In [8]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7845,0.7784,0.1493,0.163321883,0.0818347,0.099977,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.01,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7928,0.7959,0.1411,0.150243412,0.0730478,0.0846062,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.01,0.02,0.01


#### Results with in/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.785,0.7786,0.1492,0.163318723,0.0817267,0.0999732,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7918,0.7952,0.1416,0.150658672,0.0735962,0.0850746,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.01,0.02,0.01


#### Results with out/out-of-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.785,0.7787,0.1493,0.163316186,0.08174,0.0999701,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7926,0.7959,0.1413,0.150363818,0.0732247,0.0847419,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.01,0.02,0.02


#### Results with out/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7843,0.7783,0.1494,0.163371303,0.0819343,0.1000375,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.03,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7897,0.7939,0.1425,0.151287926,0.0744723,0.0857867,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.02


#### Results with anti/anti-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7841,0.7784,0.1496,0.163415108,0.0821006,0.1000912,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.02


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7921,0.7959,0.1413,0.150302539,0.0733129,0.0846728,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.0


#### Results with out/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.785,0.7787,0.1493,0.163335835,0.0817651,0.0999941,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.03,0.03,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7928,0.796,0.1411,0.150214097,0.0730772,0.0845732,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.01


#### Results with anti/in-phase initial condition

In [9]:
# k = 0.00002475 # 
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.785,0.7789,0.1494,0.163365238,0.0818684,0.1000301,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7932,0.7965,0.1411,0.150119288,0.0730117,0.0844665,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.03,0.02,0.0


#### Results with anti/out-of-phase initial condition

In [9]:
# k = 0.00002475 # This should be for y, not for z
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0, 
                           kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8212,0.7981,0.1149,0.126325405,0.0516885,0.0560405,0.067,0.001,8.89e-05,0.65,270,430,412.8,2.475e-05,10,0.09,0.2,991,0.3,0,0.02,0.02,0.01


In [9]:
# k = 0.000099 #
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, f = f0, 
                            alfa = alfa0, beta = beta0, betap = betap0, kappa = kappa0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,kappa,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.793,0.7963,0.1412,0.150282923,0.0731661,0.0846507,0.067,0.001,8.89e-05,0.65,270,430,412.8,9.9e-05,10,0.3,0.1,991,0.3,0,0.02,0.02,0.01


## Results for uncoupled oscillators
### Variable $x$

#### Results with in/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9059,0.8917,0.0968,0.104246138,0.0328673,0.0337309,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with in/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9054,0.891,0.097,0.104645087,0.0330488,0.0339896,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with in/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9094,0.8965,0.0949,0.101619069,0.0316354,0.0320523,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.01,0.02


#### Results with out/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.907,0.8928,0.0962,0.103655687,0.0324681,0.0333499,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.04,0.01


#### Results with out/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.906,0.8916,0.0967,0.104286669,0.0328339,0.0337571,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.02,0.02,0.0


#### Results with anti/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.907,0.8927,0.0962,0.103703435,0.0324889,0.0333806,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.02,0.03,0.0


#### Results with out/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9083,0.8942,0.0955,0.102907153,0.0320376,0.03287,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.02,0.02,0.01


#### Results with anti/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9084,0.8942,0.0955,0.102890136,0.0319929,0.0328591,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with anti/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9079,0.8937,0.0957,0.103169382,0.0321611,0.0330377,0.067,0.001,0.0,0.65,270,430,412.8,10,0.08,0.2,991,0.3,0,0.01,0.03,0.01


### Variable $y$

#### Results with in/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8184,0.7958,0.1158,0.127110778,0.0525351,0.0567395,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with in/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8176,0.7945,0.1162,0.127655431,0.0528375,0.0572268,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with in/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8247,0.8048,0.1133,0.123419157,0.0502797,0.0534916,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.02,0.02,0.02


#### Results with out/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8197,0.7969,0.1154,0.126765681,0.0521297,0.0564318,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.02,0.04,0.01


#### Results with out/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8184,0.7953,0.1159,0.127359661,0.0525826,0.0569619,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.02,0.02,0.0


#### Results with anti/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8197,0.7967,0.1154,0.126865604,0.0521684,0.0565208,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.01,0.01,0.0


#### Results with out/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8213,0.7988,0.1148,0.126008635,0.0515718,0.0557598,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.01,0.02,0.0


#### Results with anti/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8214,0.7986,0.1148,0.126139612,0.0515782,0.0558758,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.01,0.02,0.01


#### Results with anti/out-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8208,0.7979,0.115,0.126392309,0.0517802,0.0560999,0.067,0.001,0.0,0.65,270,430,412.8,10,0.09,0.2,991,0.3,0,0.03,0.03,0.01


### Variable $z$

#### Results with in/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7941,0.7842,0.1502,0.1662102,0.0827962,0.1035445,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with in/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7941,0.7843,0.1503,0.166226918,0.0828855,0.1035653,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with in/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.794,0.7841,0.1502,0.166207428,0.0827904,0.103541,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.03,0.01


#### Results with out/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7939,0.784,0.1502,0.166210407,0.0828164,0.1035447,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with out/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7939,0.7842,0.1503,0.166227061,0.0829087,0.1035655,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.02,0.02,0.01


#### Results with anti/anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7934,0.7839,0.1504,0.16623194,0.083013,0.1035715,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with out/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7938,0.784,0.1502,0.166207781,0.08281,0.1035414,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.01


#### Results with anti/in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7934,0.7837,0.1503,0.166211901,0.0828936,0.1035466,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.01,0.02,0.0


#### Results with anti/out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0, betap = betap0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,betap,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7935,0.7838,0.1503,0.166214381,0.0829029,0.1035497,0.067,0.001,0.0,0.65,270,430,412.8,10,0.4,0.0,991,0.3,0,0.02,0.03,0.0


## Results for single oscillator
### Variable $x$
#### Results with in-phase initial condition

In [10]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,nan,nan,nan,nan,nan,nan,0.067,0.001,0.0,0.65,270,430,5,0.01,0.0,8910,0.2,0,1.0,0.01,0.29


#### Results with out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8822,0.8592,0.1101,0.12009659,0.042575,0.0447682,0.067,0.001,0.0,0.65,270,430,5,0.04,0.2,991,0.3,0,0.0,0.02,0.0


#### Results with anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8803,0.8543,0.1112,0.122488367,0.043367,0.0465691,0.067,0.001,0.0,0.65,270,430,5,0.04,0.2,991,0.3,0,0.0,0.02,0.0


### Variable $y$
#### Results with in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.8195,0.7992,0.1157,0.125376922,0.052372,0.0552021,0.067,0.001,0.0,0.65,270,430,5,0.04,0.2,991,0.3,0,0.0,0.02,0.0


#### Results with out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7961,0.764,0.1243,0.136592196,0.0605419,0.0655198,0.067,0.001,0.0,0.65,270,430,5,0.04,0.2,991,0.3,0,0.0,0.02,0.0


#### Results with anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7942,0.7598,0.1249,0.137955285,0.0611058,0.066834,0.067,0.001,0.0,0.65,270,430,5,0.04,0.2,991,0.3,0,0.0,0.02,0.0


### Variable $z$
#### Results with in-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.9978,0.9978,0.0163,0.016360114,0.0010266,0.0010503,0.067,0.001,0.0,0.65,270,430,5,0.4,0.3,8910,0.2,0,0.76,0.03,0.22


#### Results with out-of-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.7798,0.77,0.1521,0.167953179,0.084868,0.1057275,0.067,0.001,0.0,0.65,270,430,5,0.4,0.0,991,0.3,0,0.0,0.02,0.0


#### Results with anti-phase initial condition

In [9]:
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.779,0.7696,0.1523,0.168009991,0.0850967,0.105799,0.067,0.001,0.0,0.65,270,430,5,0.4,0.0,991,0.3,0,0.01,0.02,0.0


## How to save results? 

In [9]:
### Simple Test ###
lrc = lrt.Reservoir(network_name = "BZ Reaction")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(epsilon = epsilon0, q = q0, gamma = gamma0, 
                                 f = f0, alfa = alfa0, beta = beta0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,epsilon,q,gamma,f,alfa,beta,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,BZ Reaction,0.998,0.998,0.0156,0.015834302,0.0009496,0.0009838,0.067,0.001,0.0,0.65,270,430,5,0.4,0.0,8910,0.2,0,0.1,0.03,0.02


In [25]:
float(lrc.result_summary_df["RMSE_test"][0])

0.015834302

In [10]:
# Save result to pickel###
import os; output_dir = "ThesisResults";
joblib.dump(lrc, "%s/LR_single_z.jb" % output_dir, compress=3)

['ThesisResults/LR_single_z.jb']

##### Results with simple regression

In [26]:
# ----------------------------------------------- #
# Perform simple ridge regression under the same condition
# ----------------------------------------------- #
pred_wo_reservoir = lrt.Reservoir(network_name = "BZ Reaction")
pred_wo_reservoir.prepare_target_data(target_ts, target_var, test_fraction)
pred_wo_reservoir.learn_model_wo_reservoir(ridge_lambda = 0.05)
pred_wo_reservoir.predict_wo_reservoir()

In [30]:
pred_wo_reservoir.test_rmse

0.02108426276863072

In [12]:
import os; output_dir = "ThesisResults";

In [13]:
# Save result to pickel####
joblib.dump(pred_wo_reservoir, "%s/LR_OLS_z.jb" % output_dir, compress=3)

['ThesisResults/LR_OLS_z.jb']